In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [5]:
# Read data

df = pd.read_csv('data.csv', index_col=0, nrows=100)
df.index = np.arange(0, len(df))
willbepredicted = pd.read_csv('data.csv', index_col=0, skiprows=range(1, 101), nrows=20)

feature_cols = ['x1', 'x2', 'x3', 'x4', 'x5']

X = df[feature_cols]
y = df.Y
x_find = willbepredicted[feature_cols]



poly = PolynomialFeatures(3, include_bias=False)


X_poly = poly.fit_transform(X)
X_poly_feature_name = poly.get_feature_names(['x'+str(l) for l in range(1,6)])
x_find = poly.fit_transform(x_find)
X_find_feature_name = poly.get_feature_names(['x'+str(l) for l in range(1,6)])
print(X_poly_feature_name)
print(len(X_poly_feature_name))

['x1', 'x2', 'x3', 'x4', 'x5', 'x1^2', 'x1 x2', 'x1 x3', 'x1 x4', 'x1 x5', 'x2^2', 'x2 x3', 'x2 x4', 'x2 x5', 'x3^2', 'x3 x4', 'x3 x5', 'x4^2', 'x4 x5', 'x5^2', 'x1^3', 'x1^2 x2', 'x1^2 x3', 'x1^2 x4', 'x1^2 x5', 'x1 x2^2', 'x1 x2 x3', 'x1 x2 x4', 'x1 x2 x5', 'x1 x3^2', 'x1 x3 x4', 'x1 x3 x5', 'x1 x4^2', 'x1 x4 x5', 'x1 x5^2', 'x2^3', 'x2^2 x3', 'x2^2 x4', 'x2^2 x5', 'x2 x3^2', 'x2 x3 x4', 'x2 x3 x5', 'x2 x4^2', 'x2 x4 x5', 'x2 x5^2', 'x3^3', 'x3^2 x4', 'x3^2 x5', 'x3 x4^2', 'x3 x4 x5', 'x3 x5^2', 'x4^3', 'x4^2 x5', 'x4 x5^2', 'x5^3']
55


In [6]:
# polynomial dataframes
df_poly = pd.DataFrame(X_poly, columns=X_poly_feature_name)
df_poly.head()

df_poly['y']=df['Y']
df_poly.head()

X_train=df_poly.drop('y',axis=1)
y_train=df_poly['y']

In [7]:
# Polynomial model without regularization and cross-validation


poly = LinearRegression(normalize=True)

model_poly=poly.fit(X_train,y_train)
y_poly = poly.predict(X_train)
RMSE_poly=np.sqrt(np.sum(np.square(y_poly-y_train)))
print("Root-mean-square error of simple polynomial model:",RMSE_poly)


coeff_poly = pd.DataFrame(model_poly.coef_,index=df_poly.drop('y',axis=1).columns, 
                          columns=['Coefficients polynomial model'])
coeff_poly

print ("R2 value of simple polynomial model:",model_poly.score(X_train,y_train))


Root-mean-square error of simple polynomial model: 5943.598869513483
R2 value of simple polynomial model: 0.7813898787585661


In [ ]:
# %% Metamodel (Optislang style :) - polynomial model with cross-validation and LASSO regularization

from sklearn.linear_model import LassoCV


model1 = LassoCV(cv=10,verbose=0,normalize=True,eps=0.001,n_alphas=100, tol=0.0001,max_iter=100000)


model1.fit(X_train,y_train)

y_pred1 = np.array(model1.predict(X_train))

y_find = np.array(model1.predict(x_find))

RMSE_1=np.sqrt(np.sum(np.square(y_pred1-y_train)))
print("Root-mean-square error of Metamodel:",RMSE_1)


coeff1 = pd.DataFrame(model1.coef_,index=df_poly.drop('y',axis=1).columns, columns=['Coefficients Metamodel'])

coeff1[coeff1['Coefficients Metamodel']!=0]